# Example showing updates to `ami_mtl`
## As of 2021-02-26

Below here I just quickly go through some code snippets

## 1. Preamble

Just for testing we set the arguments with `sys`

In [1]:
import sys
sys.argv = 'test.py --config=/data/jwst_ami/data/neil_wrap/inputs/example.yaml'.split()

This stuff will be in our script

In [2]:
from ami_mtl.core.core import param_functions
from ami_mtl.core.core import log_functions
from ami_mtl.core.instrument import constants
from ami_mtl.science import wrap

# =============================================================================
# Define variables
# =============================================================================
# set name
__NAME__ = 'ami_mtl_wrapper.py'
__DESCRIPTION__ = ('wrapper around simulation + extraction + analysis codes '
                   'for JWST AMI mode')
# get default constants
consts = constants.Consts
# copy for update
lconsts = consts.copy(__NAME__)
# set very basic constants
__VERSION__ = lconsts.constants['PACKAGE_VERSION'].value
__DATE__ = lconsts.constants['PACKAGE_VERSION_DATE'].value
# set up the logger (for notebook printing we use log override)
log = log_functions.Log(log_override=True)
# define group name
group = 'recipe'

# =============================================================================
# Define arguments
# =============================================================================
# Define the config file
lconsts.add_argument('WCONFIG', value=None, dtype=str,
                     source=__NAME__, user=True, argument=True,
                     group=group,
                     description='Define the target as it appears in the '
                                 'APT file',
                     command=['--config'])

## 2. Main code snippets

1. Load parameters (from constants files)
2. Load simulations

In [3]:
# Load parameters (quiet mode for notebook only)
params = param_functions.setup(lconsts, dict(), log=log,
                               desc=__DESCRIPTION__, name=__NAME__,
                               quiet=True)

# note this is for me (print not working in pycharm juptyer notebook)
_print = params.log.info

Now we can look at what we have (with a nice printout of params)
note these are **NOT** from the yaml file

In [4]:
params


ParamDict:

 	DEBUG:                        0                                             # core.instruments.constants.py
 	USER_CONFIG_FILE:             None                                          # core.instruments.constants.py
 	PACKAGE_NAME:                 ami_mtl                                       # core.instruments.constants.py
 	PACKAGE_VERSION:              0.0.006                                       # core.instruments.constants.py
 	PACKAGE_VERSION_DATE:         2021-02-17                                    # core.instruments.constants.py
 	PACKAGE_THEME:                DARK                                          # core.instruments.constants.py
 	PACKAGE_DIRECTORY:            ami_sim_data                                  # core.instruments.constants.py
 	GENERATE_CONFIG_FILE:         False                                         # core.instruments.constants.py
 	DIRECTORY:                    .                                             # core.instruments.constants.p

## 2.1 The Yaml file

Let's look at the yaml file (loaded in `params['WCONFIG']`)

You'll see its just a dictionary of dictionarys (like any other standard yaml file)


In [5]:
import yaml

with open(str(params['WCONFIG'])) as yfile:
    properties = yaml.load(yfile, Loader=yaml.FullLoader)


properties

{'Simulation1': {'name': 'Simulation 1',
  'xmlpath': '/data/jwst_ami/data/neil_wrap/inputs/jwst_ami_p23_example.xml',
  'filters': ['F380M', 'F480M'],
  'target': {'name': 'HD-218396',
   'ra': 'None',
   'dec': 'None',
   'pmra': 'None',
   'pmdec': 'None',
   'magnitude': {'F277W': 12.123,
    'F380M': 12.123,
    'F430M': 12.123,
    'F480M': 12.123},
   'companion1': {'kind': 'planet',
    'name': 'planet1',
    'separation': 0.5,
    'pa': 90,
    'dmag': {'F277W': 20.123,
     'F380M': 20.123,
     'F430M': 20.123,
     'F480M': 20.123}},
   'companion2': {'kind': 'planet',
    'name': 'planet2',
    'separation': 1.0,
    'pa': 180,
    'dmag': {'F277W': 20.123,
     'F380M': 20.123,
     'F430M': 20.123,
     'F480M': 20.123}}},
  'calibrator1': {'name': 'HD-218172',
   'tot_exp': 'None',
   'ext_flux': 'None',
   'magnitude': {'F277W': 10.123,
    'F380M': 10.123,
    'F430M': 10.123,
    'F480M': 10.123}}},
 'parameters': {'general': {'in_dir': '/data/jwst_ami/data/neil_wrap


## 2.2 The Simulation class
Now we load the simulations

In [6]:
simulations = wrap.load_simulations(params, str(params['WCONFIG']))

Reading XML: /data/jwst_ami/data/neil_wrap/inputs/jwst_ami_p23_example.xml


We have a list of `Simulation` classes, lets look at the first simulation

In [7]:
# print the length
_print('Length of simulations: {0}'.format(len(simulations)))

Length of simulations: 1


In [18]:
# get the first simulation
simulation1 = simulations[0]

# look at the attributes a Simulation has
_print('Simulation name: {0}'.format(simulation1.name))
for key in simulation1.__dict__:
    _print('\tkey = {0}'.format(key))

Simulation name: SIMULATION_1
        key = name
        key = params
        key = xmlpath
        key = use_filters
        key = target
        key = calibrators


Now we can look at some of these in more detail:

- this is the xml file path that was loaded

In [10]:
_print('Simulation xmlpath: {0}'.format(simulation1.xmlpath))

Simulation xmlpath:
	/data/jwst_ami/data/neil_wrap/inputs/jwst_ami_p23_example.xml


- this is the simulation filters that are to be used

In [11]:
_print('Simulation filters: {0}'.format(simulation1.use_filters))

Simulation filters: ['F380M', 'F480M']


- these are the two classes that hold the target and calibrators:

In [12]:
_print('Target: {0}'.format(simulation1.target))

_print('Calibrators: {0}'.format(simulation1.calibrators))

Target: Target[HD_218396]
Calibrators: [Calibrator[HD_218172]]


## 2.3 The Target class

In [13]:
target = simulation1.target

look at the attributes the target has

In [17]:
_print('Target name: {0}'.format(target.name))
for key in target.__dict__:
    _print('\tkey = {0}'.format(key))

Target name: HD_218396
        key = params
        key = properties
        key = name
        key = raw_magnitudes
        key = magnitudes
        key = companions


where the companions list contains information about the companions:

In [15]:
for companion in target.companions:
    _print('Companion name: {0}'.format(companion.name))
    for key in companion.__dict__:
        _print('\tkey = {0}'.format(key))

Companion name: PLANET1
        key = params
        key = properties
        key = name
        key = raw_magnitudes
        key = magnitudes
        key = kind
        key = separation
        key = position_angle
Companion name: PLANET2
        key = params
        key = properties
        key = name
        key = raw_magnitudes
        key = magnitudes
        key = kind
        key = separation
        key = position_angle


## 2.3 The Calibrator class

In [ ]:
target = simulation1.target

look at the attributes each calibrator has:

In [19]:
for calibrator in simulation1.calibrators:

    _print('Calibrator name: {0}'.format(target.name))
    for key in target.__dict__:
        _print('\tkey = {0}'.format(key))


Calibrator name: HD_218396
        key = params
        key = properties
        key = name
        key = raw_magnitudes
        key = magnitudes
        key = companions


# 3. How we use the simulation class

## 3.1 Simulation

Using the simluation class

In [ ]:
# >> simulations = wrap.sim_module(simulations)

Runs simulations for AMI-SIM for target and for calibrators
using `simulation.params['AMISIM-USE']` (missing some functionality)

Runs (unfinished) Mirage for target and for calibrators
using `simulation.params['MIRAGE-USE']`

## 3.2 DMS

In [ ]:
# >> simulations = wrap.dms_module(simulations)

## 3.3 Extraction

In [ ]:
# >> simulations = wrap.extract_module(simulations)

## 3.4 Analysis

In [ ]:
# >> simulations = wrap.analysis_modeul(simulations)

# 4. Currently missing

1. Need function to go from `mag[fitler]` to `flux [e-/s]`
2. Need function to go from `dmag[filter]` to `contrast`

Then the simulation (AMI-SIM) could work

3. Add in dms routines (take simulations through jwst pipeline)
4. Add in extraction (AMI-CAL and or ImPlaneIA)
5. Add in analysis (AMI-CAL and or ImPlaneIA)